In [1]:
import shutil
import pandas as pd
import re

In [2]:
nonLa_image_metadata = pd.read_csv('../../data_samples/results/processed_subset_results/local_nonLa_art.csv')

### TODO: Fix Current Issue / Duplicate image names and authors are in the metadata, and during download I was only able to reach one, as the duplicates were all overwritten. (one name/many images) Complete
### TODO: Replace naming convention for file names by appending the objectID tag which will fully resolve issue with duplicate rows. There may also be a sampling issue during the creation of the nonLA image sample which caused the data to be sampled with replacement which potentially caused so many redundant rows and images during downloading which increased the load time.

In [7]:
nonLa_image_metadata.columns

Index(['objectid', 'title', 'medium', 'dimensions', 'inscription', 'markings',
       'attributioninverted', 'attribution', 'provenancetext', 'creditline',
       'classification', 'parentid', 'isvirtual', 'portfolio', 'series',
       'sequence', 'width', 'height', 'maxpixels', 'depictstmsobjectid',
       'assistivetext', 'constituentid', 'displayorder', 'ulanid',
       'preferreddisplayname', 'forwarddisplayname', 'lastname', 'nationality',
       'constituenttype', 'expanded_url', 'percent_fromCountry',
       'Country Name', 'Country', 'Continent', 'file_name', 'directory',
       'file_downloaded'],
      dtype='object')

### Identifying the duplicates which causes a reduction in the num of images downloaded due to file name overwriting issue

In [11]:
#these are files we need to reinspect due to their multiplicity
series_num_objs = nonLa_image_metadata.loc[:,['title', 'lastname', 'preferreddisplayname', 'displayorder']].groupby(['title', 'lastname']).apply(len)

In [25]:
series = series_num_objs[series_num_objs > 2]

In [39]:
series_groups = series.index

In [41]:
duplicates = None
for title, author in series_groups:
    dupe_partition = nonLa_image_metadata.where((nonLa_image_metadata.title == title) & (nonLa_image_metadata.lastname == author) ).dropna(how='all')
    if duplicates is None:
        duplicates = dupe_partition
    else:
        duplicates = pd.concat([duplicates, dupe_partition])

In [46]:
duplicates = duplicates.reset_index(drop=False)

In [61]:
unique_from_duplicates = duplicates.groupby(['title', 'lastname', 'expanded_url'])

In [82]:
unique = None

In [69]:
nonLa_image_metadata.shape

(1953, 37)

In [83]:
for art_info, data in unique_from_duplicates:
    single_pick = data.iloc[0,:]
    to_trash = list(data.iloc[1:, :].index)
    #collect all the unique datas into unique df for future download
    if unique is None:
        unique = pd.DataFrame(single_pick)
    else:
        unique = pd.concat([unique, single_pick], axis=1)
    #remove all duplicates (except for one) from the nonLa image metadata set
    #nonLa_image_metadata = nonLa_image_metadata.drop(to_trash)

In [88]:
to_download = unique.T

In [90]:
to_download = to_download.reset_index(drop=True)

In [92]:
to_download

,index,objectid,title,medium,dimensions,inscription,markings,attributioninverted,attribution,provenancetext,...,nationality,constituenttype,expanded_url,percent_fromCountry,Country Name,Country,Continent,file_name,directory,file_downloaded
0,184,6829.0,"""Ah Mr. Hardy, Mr. Hardy, if you only knew all...",drypoint,NaN,NaN,NaN,"Dyson, Will",Will Dyson,NaN,...,Australian,individual,https://api.nga.gov/iiif/9b625788-593a-4e9a-9c...,0.000356,Australia,AU,Oceania,"""Ah_Mr._Hardy,_Mr._Hardy,_if_you_only_knew_all...",../../non_laImages,True
1,266,1692.0,"""Beehive"" Water Pot",porcelain with peachbloom glaze,overall: 8.3 x 12.7 cm (3 1/4 x 5 in.),in standard script on the base in underglaze b...,NaN,Chinese Qing Dynasty,Chinese Qing Dynasty,"Thomas B. Clarke [1848-1913], New York; sold 1...",...,Chinese,anonymous,https://api.nga.gov/iiif/1339e031-ef63-4ab5-b7...,0.001907,China,CN,Asia,"""Beehive""_Water_Pot_Chinese_Qing_Dynasty.jpg",../../non_laImages,True
2,1909,53322.0,"""Beehive"" Water Pot",porcelain with peachbloom glaze,overall: 8.8 x 12.7 cm (3 7/16 x 5 in.),in standard script on the base in underglaze b...,NaN,Chinese Qing Dynasty,Chinese Qing Dynasty,"Harry G. Steele [1881-1942], Pasadena; his wid...",...,Chinese,anonymous,https://api.nga.gov/iiif/2f7e8059-3544-4a57-80...,0.001907,China,CN,Asia,"""Beehive""_Water_Pot_Chinese_Qing_Dynasty.jpg",../../non_laImages,True
3,1710,1693.0,"""Beehive"" Water Pot",porcelain with peachbloom glaze,overall: 89 x 12.5 cm (35 1/16 x 4 15/16 in.),in standard script on the base in underglaze b...,NaN,Chinese Qing Dynasty,Chinese Qing Dynasty,Possibly collection of Lord Kitchener.[1] Thom...,...,Chinese,anonymous,https://api.nga.gov/iiif/b0b56671-265b-4127-90...,0.001907,China,CN,Asia,"""Beehive""_Water_Pot_Chinese_Qing_Dynasty.jpg",../../non_laImages,True
4,384,1693.0,"""Beehive"" Water Pot",porcelain with peachbloom glaze,overall: 89 x 12.5 cm (35 1/16 x 4 15/16 in.),in standard script on the base in underglaze b...,NaN,Chinese Qing Dynasty,Chinese Qing Dynasty,Possibly collection of Lord Kitchener.[1] Thom...,...,Chinese,anonymous,https://api.nga.gov/iiif/b8892c30-3991-4cef-8b...,0.001907,China,CN,Asia,"""Beehive""_Water_Pot_Chinese_Qing_Dynasty.jpg",../../non_laImages,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,466,1750.0,Wine Cup,porcelain with famille verte and famille jaune...,overall (without handle): 3.2 x 5.6 cm (1 1/4 ...,NaN,NaN,Chinese Qing Dynasty,Chinese Qing Dynasty,"J. Pierpont Morgan [1837-1913], New York, by 1...",...,Chinese,anonymous,https://api.nga.gov/iiif/f50a7b19-f48c-4e0f-85...,0.001907,China,CN,Asia,Wine_Cup_Chinese_Qing_Dynasty.jpg,../../non_laImages,True
220,288,1740.0,Wine Pot,"porcelain with famille verte, famille jaune, a...",overall (with lid): 11.4 x 15.2 cm (4 1/2 x 6 ...,NaN,NaN,Chinese Qing Dynasty,Chinese Qing Dynasty,"J. Pierpont Morgan [1837-1913], New York, by 1...",...,Chinese,anonymous,https://api.nga.gov/iiif/a22c71b9-0e37-4c2d-9d...,0.001907,China,CN,Asia,Wine_Pot_Chinese_Qing_Dynasty.jpg,../../non_laImages,True
221,1470,1739.0,Wine Pot,"porcelain with famille verte, famille jaune, a...",overall (with lid): 11.3 x 15.4 cm (4 7/16 x 6...,NaN,NaN,Chinese Qing Dynasty,Chinese Qing Dynasty,"J. Pierpont Morgan [1837-1913], New York, by 1...",...,Chinese,anonymous,https://api.nga.gov/iiif/e5a7c645-b8f3-4db4-9e...,0.001907,China,CN,Asia,Wine_Pot_Chinese_Qing_Dynasty.jpg,../../non_laImages,True
222,111,169698.0,Zinnias,wood engraving in black on wove paper,image: 10.16 × 12.54 cm (4 × 4 15/16 in.)\nshe...,lower right in block: Lionel Lindsay; lower le...,stamped center left verso in black ink: F.B.B....,"Lindsay, Lionel",Lionel Lindsay,"Frank B. Bristow, Falls Church, Virginia; Corc...",...,Australian,individual,https://api.nga.gov/iiif/242f2eeb-af4f-4cd1-a8...,0.000356,Australia,AU,Oceania,Zinnias_Lindsay.jpg,../../non_laImages,True


### Redownloading the images with duplicate filenames to increase size of training and testing dataset in nonLa art

In [94]:
import shutil
import requests
def download_image(url, file_name, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(file_name, "wb") as f:
            f.write(response.content)
        shutil.move(file_name,'../../non_laImages/' + file_name)
    else:
        print(response.status_code)
#Define HTTP Headers
headers = {
    "User-Agent": "Chrome/51.0.2704.103",
}
import time
for i in range(0, len(to_download)):
    #Define URL of an image
    url = to_download.expanded_url[i]
    # Define image file name
    #check for nans in last name or title
    title = 'missing'
    last_name = 'missing'
    if isinstance(to_download.title[i], str):
        title = to_download.title[i]
    if isinstance(to_download.lastname[i], str):
        last_name = to_download.lastname[i]
    file_name = str.replace(title, ' ', '_').replace('/', '&') + '_' + str.replace(last_name, ' ', '_').replace('/', '&') + '_'  + str(int(to_download.objectid[i])) + '.jpg'
    if len(file_name[:-4]) > 250:
        num_remove = len(file_name[:-4]) - 250
        file_name = file_name[:-(4 + num_remove)] + '.jpg'
    if isinstance(last_name, str) & isinstance(title, str):
        print(last_name)
        print(title)
    # Download image
    #timer delay (15 seconds)
    time.sleep(15)
    download_image(url, file_name, headers)

Dyson
"Ah Mr. Hardy, Mr. Hardy, if you only knew all the circumstances..."
Chinese Qing Dynasty
"Beehive" Water Pot
Chinese Qing Dynasty
"Beehive" Water Pot
Chinese Qing Dynasty
"Beehive" Water Pot
Chinese Qing Dynasty
"Beehive" Water Pot
Chinese Qing Dynasty
"Beehive" Water Pot
Dyson
"Can you forgive me Father?" - "Can you forgive me daughter?"
Kentridge
A Nicely Built City Never Resists Destruction
Kentridge
Act I, Scene 2, from Ubu Tells the Truth
Kentridge
Act II, Scene 1, from Ubu Tells the Truth
Kentridge
Act II, Scene 5, from Ubu Tells the Truth
Kentridge
Act III, Scene 4, from Ubu Tells the Truth
Kentridge
Act III, Scene 9, from Ubu Tells the Truth
Kentridge
Act IV, Scene 1, from Ubu Tells the Truth
Kentridge
Act IV, Scene 7, from Ubu Tells the Truth
Kentridge
Act V, Scene 4, from Ubu Tells the Truth
Chinese Qing Dynasty
Amphora Vase
Chinese Qing Dynasty
Amphora Vase
Chinese Qing Dynasty
Amphora Vase
Chinese Qing Dynasty
Amphora Vase
Chinese Qing Dynasty
Amphora Vase
Chinese Qi

### Splitting the Non La Art, for training and testing

In [5]:
from sklearn.model_selection import train_test_split
#1580 rows for training (0.008%) & 396 (0.002%) for validation/testing == 1976 for one iteration of sample (1/100 of total dataset)
X_train_sample, X_valid_sample, y_train_sample, y_valid_sample = train_test_split(nonLa_image_metadata, nonLa_image_metadata.title, train_size=0.8,test_size=0.2, random_state=626)

### Making the individual directories for testing and training
For Non-Latin American art